In [1]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [2]:
len(text)

1115394

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])
print(encode("hill there"))
print(decode(encode("hill there")))

[46, 47, 50, 50, 1, 58, 46, 43, 56, 43]
hill there


In [5]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)

torch.Size([1115394]) torch.int64


In [6]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,),)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("-----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} thr target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] thr target: 43
when input is [24, 43] thr target: 58
when input is [24, 43, 58] thr target: 5
when input is [24, 43, 58, 5] thr target: 57
when input is [24, 43, 58, 5, 57] thr target: 1
when input is [24, 43, 58, 5, 57, 1] thr target: 46
when input is [24, 43, 58, 5, 57, 1, 46] thr target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] thr target: 39
when input is [44] thr target: 53
when input is [44, 53] thr target: 56
when input is [44, 53, 56] thr target: 1
when input is [44, 53, 56, 1] thr target: 58
when input is [44, 53, 56, 1, 58] thr target: 46
when input is [44, 5

In [8]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [9]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx,targets=None):
        logits = self.token_embedding_table(idx)   #(B,T,C)
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss

    def generate(self,idx,max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits,loss = self(idx)
            logits = logits[:,-1,:] #(B,C)
            probs = F.softmax(logits,dim=-1) #(B,C)
            idx_next = torch.multinomial(probs,num_samples=1)#(B,1)
            idx = torch.cat((idx,idx_next),dim=1)#(B,T+1)
        return idx
m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1),dtype=torch.long)
max_new_tokens = 100
print(decode(m.generate(idx,max_new_tokens)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [10]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [11]:
batch_size = 32
for steps in range(5000):
    xb,yb = get_batch('train')
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.6809186935424805


In [12]:
print(decode(m.generate(idx,max_new_tokens=300)[0].tolist()))


Mavanghtang d wsramangT:
D: nd TINT:
Fof;AURKIfediTus, be:ghere
Dyo.O'd, t fed inks cerVis benes. owepwnof pre, athar wowir W:
ALI u
Bue;bOgfatho rend thic; is be wara!
PO.
WGListh R.
JO:
TERI! ce, ince:'
t.
Sin honk$zlerseestindovrer wat boue nodgh flle, mOM:

SCKpoto AKIFwinthind d me sesete gkerw


### The mathematical trick in self-attention

In [13]:
#consider the following toy example:

torch.manual_seed(1337)
B,T,C=4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [14]:
#We want x[b,t] = mean_{i<=t} x[b,i]
# version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev,0)

In [15]:
x[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-6.6310e-01, -2.5128e-01,  1.0101e+00,  1.2155e-01,  1.5840e-01,
          1.1340e+00, -1.1539e+00, -2.9840e-01, -5.0754e-01, -9.2392e-01,
          5.4671e-01, -1.4948e+00, -1.2057e+00,  5.7182e-01, -5.9735e-01,
         -6.9368e-01,  1.6455e+00, -8.0299e-01,  1.3514e+00, -2.7592e-01,
         -1.5108e+00,  2.1048e+00,  2.7630e+00, -1.7465e+00,  1.4516e+00,
         -1.5103e+00,  8.2115e-01, -2.1153e-01,  7.7890e-01,  1.5333e+00,
          1.6097e+00, -4.0323e-01],
        [-8.3447e-01,  5.9780e-01, -5.14

In [16]:
xbow[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-2.4116e-01, -1.6063e-01,  3.2526e-01, -3.9683e-01,  3.9208e-01,
          5.7976e-01, -9.9692e-02, -1.1702e-01, -7.3193e-02,  1.2198e-01,
         -4.0159e-01, -1.0025e+00, -4.8488e-01,  1.6602e-01, -7.5923e-01,
          4.2481e-01,  1.4972e+00, -4.7132e-01,  8.1860e-01,  3.4460e-01,
         -1.7740e+00,  1.2990e+00,  2.1250e+00, -5.7775e-01,  7.8881e-01,
         -1.5365e+00, -1.6947e-01, -2.7318e-01,  6.1334e-01,  3.6581e-01,
          1.5667e+00,  1.0527e+00],
        [-4.3893e-01,  9.2179e-02,  1.99

In [17]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(dim=1,keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [18]:
xbow2 = wei @ x #(T,T) @ (B,T,C) ---> (B,T,T) @ (B,T,C)
torch.allclose(xbow,xbow2,atol=1e-7)

True

In [19]:
# version 3:use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))  #看作衡量交流能力的指数，负无穷代表不能和未来的元素进行对话交流
wei = F.softmax(wei,dim=-1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3,atol=1e-7)

True

In [20]:
# let's see a single head of self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x)  #(B,T,16)
q = query(x)    #(B,T,16)
wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---> (B,T,T)
v = value(x)
trril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

实际上，我们需要缩放注意力。来看下面这个例子。

In [24]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) #* (head_size**-0.5)

In [ ]:
k.var(),q.var(),wei.var()   #wei的方差达到了16，也就是head_size，这是我们在初始化时不想看到的。我们希望初始化时尽可能的平均，不至于太分散，所以要归一化

(tensor(0.9006), tensor(1.0037), tensor(15.9317))

In [26]:
wei = q @ k.transpose(-2,-1) * (head_size**-0.5)
wei.var()

tensor(0.9957)